In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [4]:
data = pd.read_csv("./data/parking_citations.corrupted.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1) 
       
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [6]:
missing_values_table(data)

,Missing Values,% of Total Values
VIN,8709705,99.8
Marked Time,8435415,96.7
Meter Id,6456512,74.0
Make,4368470,50.1
Plate Expiry Date,794827,9.1
Route,65354,0.7
Body Style,8890,0.1
Fine amount,6507,0.1
Color,4115,0.0
Issue time,2583,0.0


In [7]:
data[data["VIN"].isna()==False]

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
1235876,4241125932,2014-10-31T00:00:00,805.0,NaN,NaN,CA,NaN,1ZVFT80N465228314,NaN,PA,BK,7045 LINDLEY AVE,00374,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,6402218.1,1894969.7
1235877,4241125943,2014-10-31T00:00:00,807.0,NaN,NaN,CA,201509.0,0199,NaN,PA,SL,6941 LINDLEY AVE,00374,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,6402214.7,1894293.0
1235878,4241125954,2014-10-31T00:00:00,809.0,NaN,NaN,CA,201502.0,3491,NaN,PA,BK,6621 LINDLEY AVE,00374,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,6402205.6,1892169.7
1235879,4241125965,2014-10-31T00:00:00,814.0,NaN,NaN,CA,NaN,1G1ZS58F47F170546,NaN,PA,SL,6430 RESEDA BLVD,00374,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,6399558.0,1890949.9
1235880,4241125976,2014-10-31T00:00:00,815.0,NaN,NaN,CA,201412.0,9817,TOYT,PA,GN,6430 RESEDA BLVD,00374,53.0,80.69BS,NO PARK/STREET CLEAN,73.0,6399558.0,1890949.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252333,4247320195,2014-11-03T00:00:00,1110.0,CB3067A,NaN,CA,NaN,1N4AL3AP2EN224587,NaN,PA,GY,355 4TH ST E,00667,56.0,88.13B+,METER EXP.,63.0,6485950.2,1840638.1
1252334,4247320206,2014-11-03T00:00:00,1131.0,NaN,NaN,CA,11.0,NV,NaN,VN,BN,300 LOS ANGELES ST S,00667,56.0,80.56E4+,RED ZONE,93.0,6487537.5,1840128.2
1252335,4247320221,2014-11-03T00:00:00,1153.0,LT288,NaN,CA,201509.0,7870,NaN,PA,GY,471 3RD ST E,00667,56.0,88.13B+,METER EXP.,63.0,6504955.2,1738919.6
1252336,4247320232,2014-11-03T00:00:00,1209.0,LT372A,NaN,CA,201503.0,9425,NaN,PA,SL,323 BOYD ST,00667,56.0,88.13B+,METER EXP.,63.0,6487783.2,1839600.4


In [8]:
data["VIN"].fillna("NV", inplace=True)

In [9]:
data["Marked Time"].fillna(0, inplace=True)

In [10]:
data[data["Meter Id"].isna()==False]

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
37,1109056185,2015-12-28T00:00:00,1938.0,WF55,0.0,CA,201512.0,NV,ACUR,PA,GY,100 S HARPER AV,00472,54.0,8813B,METER EXPIRED,63.0,9.999900e+04,9.999900e+04
38,1109056196,2015-12-28T00:00:00,1942.0,WF54,0.0,CA,NaN,NV,NaN,PA,GY,100 S HARPER,00472,54.0,8813B,METER EXPIRED,63.0,9.999900e+04,9.999900e+04
39,1109056200,2015-12-28T00:00:00,1951.0,WF177A,0.0,CA,201604.0,NV,NaN,PA,GY,8372 W 3RD ST,00472,54.0,8813B,METER EXPIRED,63.0,9.999900e+04,9.999900e+04
40,1109056211,2015-12-28T00:00:00,1953.0,WF99,0.0,CA,201603.0,NV,NaN,PA,WH,1005 ORLANDO,00472,54.0,8813B,METER EXPIRED,63.0,9.999900e+04,9.999900e+04
41,1109056222,2015-12-28T00:00:00,1955.0,WF100,0.0,CA,201601.0,NV,NISS,PA,SI,100 S ORLANDO,00472,54.0,8813B,METER EXPIRED,63.0,9.999900e+04,9.999900e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8726009,4347602394,2019-01-10T00:00:00,1245.0,VN686B,0.0,CA,201905.0,NV,NaN,PA,WT,14301 DELANO ST,00300,53.0,88.13B+,METER EXP.,63.0,6.427279e+06,1.889149e+06
8726010,4347602405,2019-01-10T00:00:00,1350.0,VN316,0.0,CA,201905.0,NV,HYUN,PA,SL,14500 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.425963e+06,1.890254e+06
8726011,4347602416,2019-01-10T00:00:00,1354.0,VN914,0.0,CA,201912.0,NV,NISS,PA,GY,14400 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.426623e+06,1.890252e+06
8726012,4347602420,2019-01-10T00:00:00,1411.0,VN725,0.0,CA,201903.0,NV,NaN,PA,GY,6300 SYLMAR AV,00300,53.0,88.13B+,METER EXP.,63.0,6.426622e+06,1.889882e+06


In [11]:
data["Meter Id"].value_counts()

37        46582
53        30478
51        29736
13        27188
49        26537
          ...  
ED760         1
WU553         1
WW2244        1
CB2348        1
2D            1
Name: Meter Id, Length: 37823, dtype: int64

In [12]:
data["Meter Id"].fillna("NA", inplace=True)

In [13]:
data["Plate Expiry Date"].fillna(0, inplace=True)

In [14]:
data["Route"].value_counts()

00600    515410
00500    363630
00402    174775
00401    170018
00403    131820
          ...  
MQ142         1
01223         1
20S39         1
13L20         1
01876         1
Name: Route, Length: 8479, dtype: int64

In [15]:
data["Route"].fillna("00600", inplace=True)

In [16]:
data[data["Body Style"].isna()==False]

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1103341116,2015-12-21T00:00:00,1251.0,NA,0.0,CA,200304.0,NV,NaN,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
1,1103700150,2015-12-21T00:00:00,1435.0,NA,0.0,CA,201512.0,NV,NaN,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,9.999900e+04,9.999900e+04
2,1104803000,2015-12-21T00:00:00,2055.0,NA,0.0,CA,201503.0,NV,NaN,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6.439998e+06,1.802686e+06
3,1104820732,2015-12-26T00:00:00,1515.0,NA,0.0,CA,0.0,NV,NaN,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6.440041e+06,1.802686e+06
4,1105461453,2015-09-15T00:00:00,115.0,NA,0.0,CA,200316.0,NV,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,9.999900e+04,9.999900e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8726009,4347602394,2019-01-10T00:00:00,1245.0,VN686B,0.0,CA,201905.0,NV,NaN,PA,WT,14301 DELANO ST,00300,53.0,88.13B+,METER EXP.,63.0,6.427279e+06,1.889149e+06
8726010,4347602405,2019-01-10T00:00:00,1350.0,VN316,0.0,CA,201905.0,NV,HYUN,PA,SL,14500 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.425963e+06,1.890254e+06
8726011,4347602416,2019-01-10T00:00:00,1354.0,VN914,0.0,CA,201912.0,NV,NISS,PA,GY,14400 FRIAR ST,00300,53.0,88.13B+,METER EXP.,63.0,6.426623e+06,1.890252e+06
8726012,4347602420,2019-01-10T00:00:00,1411.0,VN725,0.0,CA,201903.0,NV,NaN,PA,GY,6300 SYLMAR AV,00300,53.0,88.13B+,METER EXP.,63.0,6.426622e+06,1.889882e+06


In [17]:
data["Body Style"].value_counts()

PA    7673976
PU     298897
VN     233505
TK     213271
CM     140263
       ...   
RO          1
16          1
VL          1
NH          1
SK          1
Name: Body Style, Length: 191, dtype: int64

In [18]:
data['Body Style'].fillna("PA", inplace=True)

In [19]:
data["Violation Description"].fillna("NO DESCRIPTION", inplace=True)

In [20]:
data["Fine amount"].fillna(data["Fine amount"].mean(), inplace=True)

In [21]:
data["Color"].fillna("NA", inplace=True)

In [22]:
data["Issue time"].fillna(0, inplace=True)

In [23]:
data["Location"].fillna("NO LOCATION", inplace=True)

In [24]:
data["RP State Plate"].value_counts()

CA    8127986
AZ      77649
TX      55337
NV      51944
FL      37924
       ...   
AS         16
NF         13
YU          8
|           2
ML          1
Name: RP State Plate, Length: 78, dtype: int64

In [25]:
data["RP State Plate"].fillna("CA", inplace=True)

In [26]:
data["Agency"].value_counts()

54.0    2150548
51.0    1943578
56.0    1846731
53.0    1571032
55.0     876274
1.0      189383
2.0       51700
57.0      37202
4.0       26652
58.0       9596
36.0       8028
40.0       5644
3.0        4965
11.0       3285
34.0        381
41.0        171
97.0         87
5.0          61
50.0         30
59.0         30
6.0          19
12.0         13
60.0         11
77.0          7
52.0          6
9.0           5
7.0           4
15.0          4
24.0          3
35.0          3
45.0          2
8.0           2
16.0          1
30.0          1
13.0          1
42.0          1
32.0          1
10.0          1
43.0          1
71.0          1
70.0          1
68.0          1
61.0          1
38.0          1
Name: Agency, dtype: int64

In [27]:
data["Agency"].fillna(54.0, inplace=True)

In [28]:
data["Issue Date"].value_counts()

2016-01-19T00:00:00    9830
2014-12-31T00:00:00    9605
2016-05-05T00:00:00    9366
2016-02-23T00:00:00    9339
2015-06-09T00:00:00    9275
                       ... 
2013-03-28T00:00:00       1
2012-10-04T00:00:00       1
2012-12-28T00:00:00       1
2014-05-17T00:00:00       1
2014-04-01T00:00:00       1
Name: Issue Date, Length: 1843, dtype: int64

In [29]:
data["Issue Date"].fillna("2016-01-19T00:00:00", inplace=True)

In [30]:
latitude = data["Latitude"]
longitude = data["Longitude"]
data["Latitude"].fillna("99999.0", inplace=True)
data["Longitude"].fillna("99999.0", inplace=True)

In [31]:
data.dtypes

Ticket number             object
Issue Date                object
Issue time               float64
Meter Id                  object
Marked Time              float64
RP State Plate            object
Plate Expiry Date        float64
VIN                       object
Make                      object
Body Style                object
Color                     object
Location                  object
Route                     object
Agency                   float64
Violation code            object
Violation Description     object
Fine amount              float64
Latitude                  object
Longitude                 object
dtype: object

In [32]:
le = LabelEncoder()
data["Meter Id"] = le.fit_transform(data["Meter Id"])
data["RP State Plate"] = le.fit_transform(data["RP State Plate"])
data["VIN"] = le.fit_transform(data["VIN"])
data["Body Style"] = le.fit_transform(data["Body Style"])
data["Color"] = le.fit_transform(data["Color"])
data["Route"] = le.fit_transform(data["Route"])
data["Violation code"] = le.fit_transform(data["Violation code"])
data["Violation Description"] = le.fit_transform(data["Violation Description"])
data["Location"] = le.fit_transform(data["Location"])

In [33]:
data["Ticket number"] = data["Ticket number"].apply(lambda x: str(x))

In [34]:
data["Ticket number"] = le.fit_transform(data["Ticket number"])

In [35]:
data["Issue Date"] = le.fit_transform(data["Issue Date"])

In [36]:
data.dtypes

Ticket number              int64
Issue Date                 int64
Issue time               float64
Meter Id                   int64
Marked Time              float64
RP State Plate             int64
Plate Expiry Date        float64
VIN                        int64
Make                      object
Body Style                 int64
Color                      int64
Location                   int64
Route                      int64
Agency                   float64
Violation code             int64
Violation Description      int64
Fine amount              float64
Latitude                  object
Longitude                 object
dtype: object

In [37]:
data["Latitude"] = data["Latitude"].apply(lambda x: float(x))
data["Longitude"] = data["Longitude"].apply(lambda x: float(x))

In [38]:
data.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,7914,726,1251.0,16241,0.0,7,200304.0,7491,NaN,107,35,249896,1105,1.0,110,661,50.000000,99999.0,99999.0
1,9213,726,1435.0,16241,0.0,7,201512.0,7491,NaN,170,101,1011165,3430,1.0,110,661,50.000000,99999.0,99999.0
2,12047,726,2055.0,16241,0.0,7,201503.0,7491,NaN,107,11,534826,4763,2.0,286,788,58.000000,6439997.9,1802686.4
3,12142,731,1515.0,16241,0.0,7,0.0,7491,NaN,107,101,4195,4623,2.0,0,17,70.112931,6440041.1,1802686.2
4,17669,629,115.0,16241,0.0,7,200316.0,7491,CHEV,107,11,1470154,3534,1.0,216,674,93.000000,99999.0,99999.0


In [39]:
data["Make"].value_counts().nlargest(25)

TOYT    721411
HOND    491961
FORD    382695
NISS    311324
CHEV    297076
BMW     199221
MERZ    177307
VOLK    149501
HYUN    133864
DODG    127764
LEXS    124508
KIA     101746
JEEP    100909
AUDI     84229
MAZD     79853
OTHR     72411
GMC      62391
CHRY     57317
INFI     56809
ACUR     52703
SUBA     46898
VOLV     42330
TOYO     40064
MITS     37842
CADI     34080
Name: Make, dtype: int64

In [40]:
top_25 = ["TOYT", "HOND", "FORD", "NISS", "CHEV", "BMW", "MERZ", "VOLK", "HYUN", "DODG", "LEXS", "KIA", "JEEP", "AUDI", "MAZD", "OTHR", "GMC", "CHRY", "INFI", "ACUR", "SUBA", "VOLV", "TOYO", "MITS", "CADI"]

In [41]:
y = []
for x in data["Make"]:
    if x in top_25:
        y.append(1)
    else:
        y.append(0)
    

In [42]:
print(len(y))
print(len(data["Make"]))

8726014
8726014


In [45]:
data = data.drop(["Make"], axis=1)

In [46]:
import lightgbm as lgb

In [48]:
clf = lgb.LGBMClassifier(n_jobs=-1)

In [49]:
clf.fit(data,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [50]:
clf.predict_proba(data)

array([[0.5705629 , 0.4294371 ],
       [0.52197786, 0.47802214],
       [0.59064765, 0.40935235],
       ...,
       [0.52403221, 0.47596779],
       [0.52431443, 0.47568557],
       [0.53156543, 0.46843457]])